<a href="https://colab.research.google.com/github/chanlee4345/invest/blob/main/bt_%EC%82%AC%EA%B3%84%EC%A0%88%EC%8B%9C%EC%A6%8C%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bt
!pip install finterstellar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for bt: filename=bt-0.2.9-cp37-cp37m-linux_x86_64.whl size=765317 sha256=87a98c5eb8c420d51eabf622bd15d0c5fad2cc2d7857f7758a96de8926a53e33
  Stored in directory: /root/.cache/pip/wheels/c5/1a/3f/b56e6064b766c107f25e200969161c09ca6868cbfe0eaa31e4
Successfully built bt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 5.0 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [2]:
import bt
import finterstellar as fs
import pandas as pd
%matplotlib inline

In [3]:
U = ['SPY','IEF','TLT','GLD','DBC']
initial_capital = 100000.0

In [101]:
df = pd.DataFrame()
for ticker in U:
  df[ticker] = fs.get_ohlc(ticker, start_date='2001-01-01')['Adj Close']
df = df.dropna()
df['CASH'] = 100.00
df

,SPY,IEF,TLT,GLD,DBC,CASH
Date,,,,,,
2006-02-06,91.71,55.38,55.26,56.72,22.05,100.00
2006-02-07,90.90,55.32,55.00,54.60,21.41,100.00
2006-02-08,91.72,55.28,54.87,54.83,21.32,100.00
2006-02-09,91.57,55.34,55.12,56.18,21.52,100.00
2006-02-10,91.74,55.18,54.82,54.79,21.13,100.00
...,...,...,...,...,...,...
2022-07-26,390.89,103.75,117.27,160.04,25.39,100.00
2022-07-27,401.04,103.96,116.61,161.67,25.97,100.00
2022-07-28,406.07,104.99,117.54,163.64,25.99,100.00


# all season 전략, SPY buy & hold

In [102]:
s1 = bt.Strategy('all_season',
                [bt.algos.RunMonthly(run_on_end_of_period=True),
                 bt.algos.SelectAll(),
                 bt.algos.WeighSpecified(SPY=0.3,IEF=0.15,TLT=0.4,GLD=0.075,DBC=0.075),
                 bt.algos.Rebalance(),
                 ])
bt1 = bt.Backtest(s1, df, initial_capital=initial_capital)

In [103]:
def buy_and_hold(data, name='buy_and_hold'):
  s = bt.Strategy(name, [bt.algos.RunOnce(),
                         bt.algos.SelectAll(),
                         bt.algos.WeighEqually(),
                         bt.algos.Rebalance()])
  return bt.Backtest(s, data, initial_capital=initial_capital)

In [104]:
spy = buy_and_hold(df['SPY'].to_frame(), name='spy')

# all season 시즌제
- 11월에서 4월 사이에만 사면 된다.
  - 4월말에 매도, 10월말에 매수
- 가격 변동이 없는 CASH 자산을 임의로 만들고, 5~10월에는 CASH를 100% 보유

In [233]:
class WeighAllSeasonSeason(bt.Algo):
  def __init__(self):
    super(WeighAllSeasonSeason, self).__init__()

  def __call__(self, target):
    current_month = target.universe[['CASH']].loc[target.now].name.month
    if current_month >= 10 or current_month < 4: # invest
      weights = pd.Series([0.3,0.15,0.4,0.075,0.075],
                          ['SPY','IEF','TLT','GLD','DBC'])
    else: # rest. all cash
      weights = pd.Series([1.0], ['CASH'])

    target.temp['weights'] = weights
    target.temp['selected'] = list(weights.index)

    return True

In [234]:
from bt.algos import PrintDate
s2 = bt.Strategy('all_season_season',
                [bt.algos.RunMonthly(run_on_end_of_period=True),
                 bt.algos.SelectAll(),
                 WeighAllSeasonSeason(),
                 #bt.algos.PrintTempData(),
                 bt.algos.Rebalance(),
                 ])
bt2 = bt.Backtest(s2, df, initial_capital=initial_capital)

In [235]:
res = bt.run(bt2, bt1, spy)

In [236]:
res.display_monthly_returns()

  Year    Jan    Feb    Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov    Dec     YTD
------  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  ------
  2006   0      0.17  -1.06   0.6       0      0      0      0      0      0   2.72  -1.18    1.21
  2007   0      1.65  -0.39   1.99      0      0      0      0      0      0   1.41   0.42    5.17
  2008   0.55   0.45   0.27   0.21      0      0      0      0      0      0   4.82   6.85   13.68
  2009  -8.19  -4.25   4.77  -0.47      0      0      0      0      0      0   3.93  -3.17   -7.75
  2010  -0.34   1.4    0.78   2.77      0      0      0      0      0      0  -0.68   0.93    4.92
  2011  -0.75   2.43   0.31   3.09      0      0      0      0      0      0   0.86   0.98    7.08
  2012   2.5    0.28  -1.2    1.99      0      0      0      0      0      0   0.89  -1.13    3.32
  2013   0.22   0.31   1.13   1.83      0      0      0      0      0      0  -0.8   -0.52    2.16
  2014   1

In [237]:
res.display()

Stat                 all_season_season    all_season    spy
-------------------  -------------------  ------------  ----------
Start                2006-02-05           2006-02-05    2006-02-05
End                  2022-08-01           2022-08-01    2022-08-01
Risk-free rate       0.00%                0.00%         0.00%

Total Return         69.05%               190.09%       347.78%
Daily Sharpe         0.60                 0.88          0.56
Daily Sortino        0.90                 1.41          0.86
CAGR                 3.24%                6.67%         9.52%
Max Drawdown         -13.88%              -17.15%       -55.17%
Calmar Ratio         0.23                 0.39          0.17

MTD                  0.00%                0.85%         -0.30%
3m                   0.00%                0.10%         0.13%
6m                   -8.23%               -8.15%        -8.64%
YTD                  -10.90%              -10.82%       -12.87%
1Y                   -9.50%               -8.16%  